In [ ]:
import tensorflow as tf

In [3]:
directory = "dataset"

train_data = tf.keras.utils.audio_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode="categorical",
    batch_size=4,
    sampling_rate=None,
    output_sequence_length=48000,
    ragged=False,
    shuffle=True,
    validation_split=0.2,
    subset="training",
    seed=7
)

validation_data = tf.keras.utils.audio_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode="categorical",
    batch_size=4,
    sampling_rate=None,
    output_sequence_length=48000,
    ragged=False,
    shuffle=False,
    validation_split=0.2,
    subset="validation",
    seed=7
)

Found 2193 files belonging to 29 classes.
Using 1755 files for training.
Found 2193 files belonging to 29 classes.
Using 438 files for validation.


In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(32, kernel_size=80, activation='relu', strides=16, input_shape=(48000, 1)),
    tf.keras.layers.MaxPooling1D(4),
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(4),
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(4),
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(4),
    tf.keras.layers.Flatten(),
    # Dense(256, activation='relu'),
    tf.keras.layers.Dense(29, activation='softmax', input_shape=(None, 10))
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2996, 32)          2592      
                                                                 
 max_pooling1d (MaxPooling1  (None, 749, 32)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 747, 32)           3104      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 186, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 184, 64)           6208      
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 46, 64)            0

In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(), 
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

In [7]:
model.fit(train_data, validation_data=validation_data, epochs=20)

Epoch 1/20


2023-12-17 14:59:05.909254: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2023-12-17 14:59:06.155016: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-17 14:59:06.809441: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f208ca1a280 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-17 14:59:06.809457: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-12-17 14:59:06.822525: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1702812546.913571    4083 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


439/439 [==============================] - 6s 7ms/step - loss: 2.7984 - accuracy: 0.1538 - val_loss: 2.5204 - val_accuracy: 0.3333
Epoch 2/20
439/439 [==============================] - 2s 4ms/step - loss: 1.5629 - accuracy: 0.5168 - val_loss: 1.6053 - val_accuracy: 0.5365
Epoch 3/20
439/439 [==============================] - 2s 4ms/step - loss: 0.9940 - accuracy: 0.6883 - val_loss: 1.0825 - val_accuracy: 0.6895
Epoch 4/20
439/439 [==============================] - 2s 5ms/step - loss: 0.7117 - accuracy: 0.7726 - val_loss: 0.9967 - val_accuracy: 0.7283
Epoch 5/20
439/439 [==============================] - 2s 5ms/step - loss: 0.5239 - accuracy: 0.8353 - val_loss: 0.7555 - val_accuracy: 0.7900
Epoch 6/20
439/439 [==============================] - 2s 4ms/step - loss: 0.4275 - accuracy: 0.8775 - val_loss: 0.5529 - val_accuracy: 0.8699
Epoch 7/20
439/439 [==============================] - 2s 4ms/step - loss: 0.3729 - accuracy: 0.8877 - val_loss: 0.5856 - val_accuracy: 0.8836
Epoch 8/20
439/43